# Ndrange Kernels, Local memory and Private memory in numba-dpex

### Learning Objectives

- Understand how ND-range improves parallelism over the basic implementation.
- Able to explain why limiting global memory access can be advantageous.
- Explain the differences between work-groups and work-items.


## ND-Range Kernels

Naive parallel kernel do not allow for performance optimizations at a hardware level. In these next two kernels we will utilize ND-Range kernels as a way to expresses parallelism enabling low level performance tuning by providing access to both global and local memory and mapping executions to compute units on hardware. The entire iteration space is divided into smaller groups called work-groups, work-items are organized into these work-groups and are scheduled on a single compute unit on the hardware.  Workgroup size must divide the entire ND-range size exactly in each dimension.  These sizes can all vary by hardware platform and by using the device queries below a developer can identify what is possible.  The workload must be considered to find the best mix of these values.

<img src="Assets/ndrange-subgroup.png">

The grouping of kernel executions into work-groups allows control of resource usage and load balance work distribution. The functionality of nd_range kernels is exposed via nd_range and nd_item classes. nd_range class represents a grouped execution range using global execution range and the local execution range of each work-group. nd_item class represents an individual instance of a kernel function and allows you to query for work-group range and index.



<img src="Assets/ndrange.png">

The work-group size depends on the accelerator hardware capability, so we set this size using command-line argument. Some hardware requre the matrix size to divide equally by the work-group size, we will use work-group size of 16x16 (256) by default which works for all the accelerator hardware we will be using to test, we will eventually use different work-group sizes to see how it impacts the performance.


## Shared Local Memory (SLM) Implementation

In a parallel algorithm, there is a high degree of reuse, so instead of loading values from global memory each time we can load the values into local memory and perform the computation.  This will reduce the latency of accessing the data values.  The difference between this implementation and the ND-range implementation is the reading is done from global memory in the case of ND-range each time and in this implementation they dat is loaded into local memory and then computed.  

When a work-group begins, the contents of its local memory are uninitialized, and local memory does not persist after a work-group finishes executing. Because of these properties, local memory may only be used for temporary storage while a work-group is executing.  For other devices though, such as many GPU devices, there are dedicated resources for local memory, and on these devices, communicating via local memory should perform better than communicating via global memory.

In SYCL’s memory model, local memory is a contiguous region of memory allocated per work group and is visible to all the work items in that group. Local memory is device-only and cannot be accessed from the host. From the perspective offers the device, the local memory is exposed as a contiguous array of a specific types. The maximum available local memory is hardware-specific. The SYCL local memory concept is analogous to CUDA’s shared memory concept.

Numba-dpex provides a special function numba_dpex.local.array to allocate local memory for a kernel. To simplify kernel development and accelerate communication between work-items in a work-group, SYCL defines a special local memory space specifically for communication between work-items in a work-group.

Local Address Space refers to memory objects that need to be allocated in local memory pool and are shared by all work-items of a work-group. Numba-dpex does not support passing arguments that are allocated in the local address space to @numba_dpex.kernel. Users are allowed to allocate static arrays in the local address space inside the @numba_dpex.kernel. In the example below numba_dpex.local.array(shape, dtype) is the API used to allocate a static array in the local address space:
These are used to compute an intermediate result which does not use global memory for repeated access for computation. 

Also notice that we used a barrier that helps to synchronize all of the work-items in the work-group. 

<img src="Assets/localmem.png">


## Local Memory sample
The following DPC++ code shows a local-memory implementation of matrix multiplication: Inspect code; there are no modifications necessary:
1. Inspect the following code cell and click Run (▶)to save the code to file.
2. Next, run (▶) the cell in the __Build and Run__ section following the code to compile and execute the code.

In [ ]:
%%writefile lab/local_memory_kernel.py
# Copyright 2020, 2021 Intel Corporation
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import dpctl
import numpy as np
from numba import float32

import numba_dpex as dpex


def no_arg_barrier_support():
    """
    This example demonstrates the usage of numba_dpex's ``barrier``
    intrinsic function. The ``barrier`` function is usable only inside
    a ``kernel`` and is equivalent to OpenCL's ``barrier`` function.
    """

    @dpex.kernel
    def twice(A):
        i = dpex.get_global_id(0)
        d = A[i]
        # no argument defaults to global mem fence
        dpex.barrier()
        A[i] = d * 2

    N = 10
    arr = np.arange(N).astype(np.float32)
    print(arr)

    # Use the environment variable SYCL_DEVICE_FILTER to change the default device.
    # See https://github.com/intel/llvm/blob/sycl/sycl/doc/EnvironmentVariables.md#sycl_device_filter.
    device = dpctl.select_default_device()
    print("Using device ...")
    device.print_device_info()

    with dpctl.device_context(device):
        twice[N, dpex.DEFAULT_LOCAL_SIZE](arr)

    # the output should be `arr * 2, i.e. [0, 2, 4, 6, ...]`
    print(arr)


def local_memory():
    """
    This example demonstrates the usage of numba-dpex's `local.array`
    intrinsic function. The function is used to create a static array
    allocated on the devices local address space.
    """
    blocksize = 10

    @dpex.kernel
    def reverse_array(A):
        lm = dpex.local.array(shape=10, dtype=float32)
        i = dpex.get_global_id(0)

        # preload
        lm[i] = A[i]
        # barrier local or global will both work as we only have one work group
        dpex.barrier(dpex.LOCAL_MEM_FENCE)  # local mem fence
        # write
        A[i] += lm[blocksize - 1 - i]

    arr = np.arange(blocksize).astype(np.float32)
    print(arr)

    # Use the environment variable SYCL_DEVICE_FILTER to change the default device.
    # See https://github.com/intel/llvm/blob/sycl/sycl/doc/EnvironmentVariables.md#sycl_device_filter.
    device = dpctl.select_default_device()
    print("Using device ...")
    device.print_device_info()

    with dpctl.device_context(device):
        reverse_array[blocksize, dpex.DEFAULT_LOCAL_SIZE](arr)

    # the output should be `orig[::-1] + orig, i.e. [9, 9, 9, ...]``
    print(arr)


def main():
    no_arg_barrier_support()
    local_memory()

    print("Done...")


if __name__ == "__main__":
    main()
    

#### Build and Run
Select the cell below and click run ▶ to compile and execute the code:

In [ ]:
! chmod 755 q; chmod 755 run_local_memory.sh; if [ -x "$(command -v qsub)" ]; then ./q run_local_memory.sh; else ./run_local_memory.sh; fi

### Private Address Space
Private Address Space refers to memory objects that are local to each work-item and is not shared with any other work-item. In the example below numba_dpex.private.array(shape, dtype) is the API used to allocate a static array in the private address space:

<img src="Assets/workgroup.png">

## Private Memory sample

The following code shows a Private-memory implementation of numba-dpex: Inspect code; there are no modifications necessary:
1. Inspect the following code cell and click Run (▶)to save the code to file.
2. Next, run (▶) the cell in the __Build and Run__ section following the code to compile and execute the code.

In [ ]:
%%writefile lab/private_memory_kernel.py
# SPDX-FileCopyrightText: 2020 - 2023 Intel Corporation
#
# SPDX-License-Identifier: Apache-2.0

import dpctl
import dpctl.tensor as dpt
import numpy as np
from numba import float32

import numba_dpex as ndpx


def private_memory():
    """
    This example demonstrates the usage of numba_dpex's `private.array`
    intrinsic function. The function is used to create a static array
    allocated on the devices private address space.
    """

    @ndpx.kernel
    def private_memory_kernel(A):
        memory = ndpx.private.array(shape=1, dtype=np.float32)
        i = ndpx.get_global_id(0)

        # preload
        memory[0] = i
        ndpx.barrier(ndpx.LOCAL_MEM_FENCE)  # local mem fence

        # memory will not hold correct deterministic result if it is not
        # private to each thread.
        A[i] = memory[0] * 2

    N = 4
    device = dpctl.select_default_device()

    arr = dpt.zeros(N, dtype=dpt.float32, device=device)
    orig = np.arange(N).astype(np.float32)

    print("Using device ...")
    device.print_device_info()

    global_range = ndpx.Range(N)
    local_range = ndpx.Range(N)
    private_memory_kernel[ndpx.NdRange(global_range, local_range)](arr)

    arr_out = dpt.asnumpy(arr)
    np.testing.assert_allclose(orig * 2, arr_out)
    # the output should be `orig[i] * 2, i.e. [0, 2, 4, ..]``
    print(arr_out)


def main():
    private_memory()

    print("Done...")


if __name__ == "__main__":
    main()
    

#### Build and Run
Select the cell below and click run ▶ to compile and execute the code:

In [ ]:
! chmod 755 q; chmod 755 run_private_memory.sh; if [ -x "$(command -v qsub)" ]; then ./q run_private_memory.sh; else ./run_private_memory.sh; fi

## Summary
In this module you will have learned the following:
* Implementation of Local memory in numba-dpex
* Implementation of Private memory in numba-dpex